In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr


In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN"], axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Apps < 500")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3            27037
T4             1542
T6             1216
T5             1173
T19            1065
T8              737
T7              725
T10             528
Apps < 500      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
counts_binning = application_df['CLASSIFICATION'].value_counts()
counts_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classifications = counts_binning[counts_binning>1]
counts_classifications

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Classification < 1000")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000                    17326
C2000                     6074
C1200                     4837
Classification < 1000     2261
C3000                     1918
C2100                     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Define bins
bins = [0, 10000, 50000, 100000, np.inf]
labels = ['0-10k', '10k-50k', '50k-100k', '100k+']

# Add a new column to the dataframe with the binned values
application_df['ASK_AMT_RANGE'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)

In [11]:
application_df['NAME'].value_counts()

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [12]:
names_to_replace = list(application_df['NAME'].value_counts()[application_df['NAME'].value_counts() < 5].index)
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"names<10")
application_df['NAME'].value_counts()
    

names<10                                                             19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: NAME, Length: 403, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,...,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_RANGE_0-10k,ASK_AMT_RANGE_10k-50k,ASK_AMT_RANGE_50k-100k,ASK_AMT_RANGE_100k+
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [14]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'NAME_AACE INTERNATIONAL',
       'NAME_ACE MENTOR PROGRAM OF AMERICA INC', 'NAME_ACTS MINISTRY',
       'NAME_ACTS MISSIONS',
       'NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS',
       'NAME_AIR FORCE ASSOCIATION', 'NAME_ALABAMA FEDERATION OF WOMENS CLUBS',
       ...
       'INCOME_AMT_1M-5M', 'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+',
       'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_N',
       'SPECIAL_CONSIDERATIONS_Y', 'ASK_AMT_RANGE_0-10k',
       'ASK_AMT_RANGE_10k-50k', 'ASK_AMT_RANGE_50k-100k',
       'ASK_AMT_RANGE_100k+'],
      dtype='object', length=451)

In [15]:
application_df.drop(columns=['ASK_AMT'], inplace=True)
application_df.columns

Index(['STATUS', 'IS_SUCCESSFUL', 'NAME_AACE INTERNATIONAL',
       'NAME_ACE MENTOR PROGRAM OF AMERICA INC', 'NAME_ACTS MINISTRY',
       'NAME_ACTS MISSIONS',
       'NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS',
       'NAME_AIR FORCE ASSOCIATION', 'NAME_ALABAMA FEDERATION OF WOMENS CLUBS',
       'NAME_ALABAMA TREASURE FOREST ASSOCIATION',
       ...
       'INCOME_AMT_1M-5M', 'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+',
       'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_N',
       'SPECIAL_CONSIDERATIONS_Y', 'ASK_AMT_RANGE_0-10k',
       'ASK_AMT_RANGE_10k-50k', 'ASK_AMT_RANGE_50k-100k',
       'ASK_AMT_RANGE_100k+'],
      dtype='object', length=450)

In [16]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# def create_model(hp):
#     nn_model = tf.keras.models.Sequential()
#     activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
#     nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#                                         min_value=1,
#                                         max_value=100,
#                                         step=2), activation=activation, input_dim=len(X_train_scaled[0])))
#     for i in range(hp.Int('num_layers', 1, 4)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#                                         min_value=1,
#                                         max_value=30,
#                                         step=2), activation=activation))
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
#     loss = hp.Choice('loss', ['binary_crossentropy', 'mse'])
#     optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad'])
#     nn_model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    
#     return nn_model

In [19]:
# tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=10, directory='my_dir2', project_name='intro_to_kt')

In [20]:
# tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

In [21]:
# best_hyper = tuner.get_best_hyperparameters(1)[0]
# best_hyper.values

In [22]:
# # Evaluate best model against full test data
# best_model = tuner.get_best_models(1)[0]
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  97
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 25
hidden_nodes_layer4 = 11

output_dim = 1



nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))




# Output layer
nn_model.add(tf.keras.layers.Dense(output_dim, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 97)                43650     
                                                                 
 dense_1 (Dense)             (None, 11)                1078      
                                                                 
 dense_2 (Dense)             (None, 25)                300       
                                                                 
 dense_3 (Dense)             (None, 11)                286       
                                                                 
 dense_4 (Dense)             (None, 1)                 12        
                                                                 
Total params: 45,326
Trainable params: 45,326
Non-trainable params: 0
_________________________________________________________________


In [40]:
nn_model.compile(loss="mse", optimizer="rmsprop", metrics=["accuracy"])

In [43]:

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200, initial_epoch= 100)

Epoch 101/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1334 - accuracy: 0.8120
Epoch 102/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1335 - accuracy: 0.8118
Epoch 103/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1332 - accuracy: 0.8119
Epoch 104/200
804/804 [==============================] - 6s 7ms/step - loss: 0.1334 - accuracy: 0.8116
Epoch 105/200
804/804 [==============================] - 5s 7ms/step - loss: 0.1333 - accuracy: 0.8123
Epoch 106/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1333 - accuracy: 0.8130
Epoch 107/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1335 - accuracy: 0.8117
Epoch 108/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1333 - accuracy: 0.8110
Epoch 109/200
804/804 [==============================] - 5s 6ms/step - loss: 0.1332 - accuracy: 0.8120
Epoch 110/200
804/804 [==============================] - 5s 6ms/step - lo

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.1480 - accuracy: 0.7913 - 705ms/epoch - 3ms/step
Loss: 0.14800231158733368, Accuracy: 0.7912536263465881


In [28]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity2.h5")